In [1]:
from datetime import datetime

import numpy as np
from scipy.optimize import least_squares

from glmtools.io.glm import GLMDataset
from glmtools.io.lightning_ellipse import ltg_ellps_radii, ltg_ellpse_rev, ltg_ellps_lon_lat_to_fixed_grid
from lmatools.grid.fixed import get_GOESR_coordsys

# Stereo renavigation GOES-R fixed grid look vectors.

This notebook demonstrates how to calclulate the stereo position of two (or more) fixed grid look vectors observed by geostationary satellites, and explains how to do it.

A common application of this approach is to navigate GOES-R GLM groups that occur near each other in space and at the same time. The event-based nature of lightning makes it easier to find matches.

We take a forward modeling, cost function minimization approach, instead of trying to calculate the nearest approach of two vectors defined by the fixed grid look vectors. This opens the methodology to three or more observations of the same event.

The practical implmentation uses the operationally tested `glmtools` functionality for coordinate system transformations, and adds new navigation code from Bezooijen et al. (2016, SPIE, [1]), hereafter B16.

— ECB, 27 November 2024

[1]     Bezooijen, R. W. H., H. Demroff, G. Burton, D. Chu, and S. Yang, 2016: 
        Image navigation and registration for the geostationary lightning 
        mapper (GLM). Proc. SPIE 10004, 100041N, doi: 10.1117/12.2242141.


## Concerning the satellite position


In B16, eq. 56 the final navigated fixed grid angles are relative to the *nominal* position of the satellite. But what is the "nominal" position?

From the PUG, L1bvol3, bottom of p. 12 (just below Table 5.1.2.3-2), -75.0 (lon_field_of_view in the L2 LCFA product) is the center of the fixed grid coordinate system for ABI, while the actual orbital slot is -75.2. 

In the PUG L2+vol5, table 5.26.1-1 on p. 589, in the section on GLM, nominal_satellite_subpoint_lon is -75.2, matching the variable in the data file with that name. 

In Bezooijen, "the nominal location of the SC is defined as the origin of the fixed grid coordinate frame," which conflicts with the two statements above and the names of the variables. One might think that B16 were also responsible for the table in the L2+ PUG, and their use of "nominal" in their equation 56 corresponds to "nominal" in the PUG, i.e. they define the fixed grid angles relative to -75.2. However, this does not seem to be the case.

Code we have previously developed has been tested for a match to LMA data, and it uses `lon_field_of_view` in a relatively low-level conversion of event positions. See `plot_glm_events` in https://github.com/deeplycloudy/xlma-python/pull/41/files. Similarly, the operational glmtools code uses `lon_field_of_view` to calculate its fixed grid positions, and this matches well to ABI. So, we interpret the fixed grid angle as being defined relative to `lon_field_of_view`.

However, it is helpful to also think about the specific steps followed by B16, who first navigate from the _true_ satellite position to a position on the lightning ellipsoid, and then use the position on the lightning ellipsoid to find a _nominal_ fixed grid angle.

B16 first determines the line of sight look angle from the true, physical GLM local, **v**\_it, a unit vector in the ECEF basis (eq. 30). This vector is then scaled with eq. 33 to the correct length to intersect the lightning ellipsoid, as calculated explicitly in section 2.6, eq. 41, which then gives the lightning position vector **P** in eq. 44 relative to ECEF. So the vector  **v**\_it is what we want, since it is determined independently of the lightning ellipsoid, and is in ECEF without reference to any earth or lightning ellipsoid.

We might take the fixed grid vectors already navigated by glmtools to be aligned with this true look angle, but is that true? Note that in B16 they calculate the look angles from the actual, physical satellite position (i.e., -75.2), find the intersection with the lightning ellipsoid, and get the ECEF vector to the lightning position **P**. Then they assume the satellite is at -75.0 (as we find to be empirically true given our other successful naviation implementations described above), and calculate that fixed grid look angle.

So to get the true look angle as it occurred in physical reality, we need to start with the calculated position on the lighting ellipsoid, and find the fixed grid angle back to the physical satellite position. The LCFA file gives the lightning ellipsoid position, so we can find the fixed grid angle to the physical satellite position with the same glmtools code by setting the nadir_lon to the satellite subpoint (-75.2).


G18 lon_field_of_view and nominal_satellite_subpoint are the same. A test using the G16 lon_field_of_view shows residuals that are twice as large, and locates lightning in the stratosphere.


## Stereo calculation methodology

Start with a first guess (lon, lat) position as the average of both satellite horizontal locations, and guess 12 km altitude (result is not sensitive to this guess). Predict the look angle using Bezooijen's (55-59), and use scipy's Levenburg-Marquardt nonlinear least squares optimization routine to adjust (lon, lat, alt) to minimize the difference to the two look angles. This produces a final lon, lat, alt of the source.


In [2]:
# === Functions for converting data in the GLM LCFA L2 files to vectors ===

def goesr_sat_params(ds):
    """ Extract satellite fixed grid field of view longtidue (lon_fov), 
        longtiude of the actual satellite subpoint (lon_subpoint),
        and distance from the center of the earth (ecef_height)
        from a GOES-R series satellite dataset ds loaded with xarray.
    """
    ecef_height = ds.nominal_satellite_height.data.astype(np.float64)*1000
    sat_ecef_height_nominal=35786023.0 # Value we expect
    assert int(ecef_height) == int(sat_ecef_height_nominal)
    
    lon_fov = ds.lon_field_of_view.data.astype(np.float64)
    lon_subpoint = ds.nominal_satellite_subpoint_lon.data.astype(np.float64)

    return lon_fov, lon_subpoint, ecef_height

class GOESR_Navigator(object):
    def __init__(self, ds):
        lon_fov, lon_subpoint, ecef_height = goesr_sat_params(ds)

        self.sat_lon = lon_subpoint
        self.ecef_height = ecef_height
        self.geofixCS, self.grs80lla = get_GOESR_coordsys(self.sat_lon)
        self.sat_vec = np.vstack(self.grs80lla.toECEF(self.sat_lon, 0.0, self.ecef_height))

In [3]:
def fixed_grid_from_obs_vec(obs_vec, sat_vec, sat_lon):
    """ Given a lightning vector and a satellite vector in ECEF coordinates,
        find the alpha and beta fixed grid angles.
        
        Implements equations 55-59 from Bezooijen (2016, SPIE).
    """
    # B16 notation
    P = obs_vec
    R = sat_vec
        
    D = P - R
    
    lon_rad = np.radians(sat_lon)
    # matrix to rotate from ITRS (ECEF) to fixed grid.
    A_it_fg = np.asarray(
                  [[-np.sin(lon_rad),  np.cos(lon_rad),  0],
                   [               0,                0, -1],
                   [-np.cos(lon_rad), -np.sin(lon_rad),  0],
                  ]
              )
    D_fg = np.matmul(A_it_fg, D)
    D_fg_norm = np.linalg.norm(D_fg)
    d1 = D_fg[0,0]/D_fg_norm
    d2 = D_fg[1,0]/D_fg_norm
    d3 = D_fg[2,0]/D_fg_norm
    alpha = -np.arctan(d2/d3)
    beta = np.arcsin(d1)
    # return as east, north angles.
    return beta, alpha

In [4]:
g16 = GLMDataset('/data/20240528-WTLMA/GLM/OR_GLM-L2-LCFA_G16_s20241492318200_e20241492318400_c20241492318416.nc')
g18 = GLMDataset('/data/20240528-WTLMA/GLM/OR_GLM-L2-LCFA_G18_s20241492318200_e20241492318400_c20241492318418.nc')

## Preliminary calculations

Are we on the right track with our implementation of arbitrary forward navigation of lon, lat, alt to a satellite?

In [5]:
g16_nav = GOESR_Navigator(g16.dataset)
g18_nav = GOESR_Navigator(g18.dataset)

ltg_lon, ltg_lat, ltg_alt = -101.5, 33.5, 12000.0

print(g16_nav.sat_vec) # seems ok as ECEF

ltg_vec = np.vstack(g16_nav.grs80lla.toECEF(ltg_lon, ltg_lat, ltg_alt))
print(ltg_vec) # seems ok as ECEF

# should be negative east (beta) and postive north (alpha)
test_b, test_a = fixed_grid_from_obs_vec(ltg_vec, g16_nav.sat_vec, g16_nav.sat_lon)
print(test_b, test_a)

[[ 10770658.09197447]
 [-40765295.89816594]
 [        0.        ]]
[[-1063443.75530698]
 [-5226993.05104585]
 [ 3506957.5318461 ]]
-0.0628625778829751 0.09353971050950552


/Users/ebruning/code/lmatools/lmatools/coordinateSystems.py:80: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  projectedData = array(proj4.transform(self.ERSlla, self.ERSxyz, lon, lat, alt ))
/Users/ebruning/code/lmatools/lmatools/coordinateSystems.py:80: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  projectedData = array(proj4.transform(self.ERSlla, self.ERSxyz, lon, lat, alt ))


In [6]:
glmtools_b, glmtools_a = ltg_ellps_lon_lat_to_fixed_grid(ltg_lon, ltg_lat, g16_nav.sat_lon, 1)

In [7]:
rad_per_km = 28e-6

In [8]:
beta_error_km =  (glmtools_b-test_b)/rad_per_km
alpha_error_km =  (glmtools_a-test_a)/rad_per_km

In [9]:
print(beta_error_km, alpha_error_km)

[0.1602367] [-0.28211879]


So we seem to be on the right track with our independent implementation of the fixed grid forward navigation from an arbitrary location!

Let's generalize our location test into a formal implementation, and use it to do a stereo retrieval.

In [10]:

def stereo_locate_residual(x, *args, **kwargs):
    """ x: array of shape (3,) for (lon, lat, alt)
    
        kwargs must contain keys for:
        goes_navs: a tuple of N instances of GOESR_Navigator objects
        goes_locs: a (2,N) array of fixed grid obsevation angles, 
            with the first dimension by convention being (beta, alpha)
            and the second dimension corresponding (in order) to the N goes_navs.

        residuals returned are m = 2*N fixed grid angle errors corresponding
        to the difference between observation locations predicted from x, 
        and the goes_locs, as pairs of (beta_error, alpha_error)
    """
    goes_navs = kwargs['goes_navs']
    goes_locs = kwargs['goes_locs']
    N = len(goes_navs)
    residuals = np.zeros(2 * N, dtype=float)
    for i, goes_nav in enumerate(goes_navs):
        this_slice = slice(i*2, (i+1)*2)
        ecef_vec = np.vstack(goes_nav.grs80lla.toECEF(x[0], x[1], x[2]))
        test_b, test_a = fixed_grid_from_obs_vec(ecef_vec, goes_nav.sat_vec, goes_nav.sat_lon)
        residuals[this_slice] = np.asarray((test_b, test_a)) - goes_locs[:, i].flatten()
    return residuals
    
def geostationary_stereo_locate(fixed_grid_locs, goes_navs, guess_alt=12000.0):
    """ Locate N independent obserations of the same physical entity
        observed from geostationary orbit.
        
        fixed_grid_locs is a (2,N) array of fixed grid obsevation angles, 
            with the first dimension by convention being (beta, alpha)
            and the second dimension corresponding (in order) to the N goes_navs.
        goes_navs: N instances of GOESR_Navigator
        
        guess_alt is an initial guess at the altitude of the observation.
    """
    
    # Convert the fixed_grid_locs into lon, lat and average to find an initial guess location
    N = len(goes_navs)
    assert fixed_grid_locs.shape[1] == N
    
    # find an initial guess from averaging the earth-intersection points
    #  of the input fixed grid locations
    initial_guess_lla = np.ones((3,N), dtype=float)*guess_alt
    for i, goes_nav in enumerate(goes_navs):
        ix, iy = fixed_grid_locs[0,i], fixed_grid_locs[1,i]
        iz = 0.0 # doesn't matter - undefined in the satellite fixed grid system
        ilon, ilat, ialt_foo = goes_nav.grs80lla.fromECEF(*goes_nav.geofixCS.toECEF(ix,iy,iz))
        initial_guess_lla[:, i] = np.vstack([ilon, ilat, guess_alt])[:,0]
    
    residual_kwargs = dict(goes_navs=goes_navs, goes_locs=fixed_grid_locs)
    # testing showed that the result with the default solver was sensitive
    # to altitude, so use ol' reliable, Levenburg-Marquardt.
    retrieval = least_squares(stereo_locate_residual, 
                              initial_guess_lla.mean(axis=1).flatten(),
                              method='lm',
                              kwargs=residual_kwargs
                             )
    retrieved_lla, fixed_grid_beta_alpha_pairs = retrieval.x, retrieval.fun
    return retrieved_lla, fixed_grid_beta_alpha_pairs


In [11]:
date = datetime(2024,5,28)

# (GLM16) 23:01:15.280388
# 	-102.177055	33.834248
glm16_event_lat = 33.834248
glm16_event_lon = -102.177055

# (GLM18) 23:01:15.279243
# 	-102.122154	33.822323
glm18_event_lat = 33.822323
glm18_event_lon = -102.122154

ellps_rev = ltg_ellpse_rev(date)

In [12]:
# Predict the location using the LCFA lon, lat location and the satellite subpoint.
x16, y16 = ltg_ellps_lon_lat_to_fixed_grid(glm16_event_lon, glm16_event_lat, g16_nav.sat_lon, ellps_rev)
x18, y18 = ltg_ellps_lon_lat_to_fixed_grid(glm18_event_lon, glm18_event_lat, g18_nav.sat_lon, ellps_rev)

fixed_grid_locs = np.asarray([[x16, x18],
                              [y16, y18],
                             ])
goes_navs = [g16_nav, g18_nav]

final_lla, final_errs = geostationary_stereo_locate(fixed_grid_locs, goes_navs, guess_alt=12000.0)
print(final_lla, final_errs)


# Predict the location using the LCFA lon, lat location and the lon_field_of_view, which differs for G16
x16, y16 = ltg_ellps_lon_lat_to_fixed_grid(glm16_event_lon, glm16_event_lat, g16_nav.sat_lon+0.2, ellps_rev)
x18, y18 = ltg_ellps_lon_lat_to_fixed_grid(glm18_event_lon, glm18_event_lat, g18_nav.sat_lon, ellps_rev)

fixed_grid_locs = np.asarray([[x16, x18],
                              [y16, y18],
                             ])
goes_navs = [g16_nav, g18_nav]

final_lla, final_errs = geostationary_stereo_locate(fixed_grid_locs, goes_navs, guess_alt=12000.0)
print(final_lla, final_errs)

[ -102.15087962    33.80652756 14336.28066174] [ 8.10395282e-07 -1.37862661e-05  8.20342874e-07  1.39235431e-05]
[ -102.26652163    33.72983352 24383.30456166] [ 9.19229035e-07 -1.56363715e-05  9.30193004e-07  1.57879083e-05]


/Users/ebruning/code/lmatools/lmatools/coordinateSystems.py:221: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return proj4.transform(self.fixedgrid, self.ECEFxyz, X, Y, Z)
/Users/ebruning/code/lmatools/lmatools/coordinateSystems.py:91: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  projectedData = array(proj4.transform(self.ERSxyz, self.ERSlla, x, y, z ))


In [13]:
date = datetime(2024,5,28)

# (GLM16) 23:18:26.343
# 	-102.182144	33.84683
glm16_event_lat = 33.84683
glm16_event_lon = -102.182144


# (GLM18) 23:18:26.342414
# 	-102.11064	33.845528
glm18_event_lat = 33.845528
glm18_event_lon = -102.11064


In [14]:
# Predict the location using the LCFA lon, lat location and the satellite subpoint.
x16, y16 = ltg_ellps_lon_lat_to_fixed_grid(glm16_event_lon, glm16_event_lat, g16_nav.sat_lon, ellps_rev)
x18, y18 = ltg_ellps_lon_lat_to_fixed_grid(glm18_event_lon, glm18_event_lat, g18_nav.sat_lon, ellps_rev)

fixed_grid_locs = np.asarray([[x16, x18],
                              [y16, y18],
                             ])
goes_navs = [g16_nav, g18_nav]

final_lla, final_errs = geostationary_stereo_locate(fixed_grid_locs, goes_navs, guess_alt=12000.0)
print(final_lla, final_errs)

# Predict the location using the LCFA lon, lat location and the lon_field_of_view, which differs for G16
x16, y16 = ltg_ellps_lon_lat_to_fixed_grid(glm16_event_lon, glm16_event_lat, g16_nav.sat_lon+0.2, ellps_rev)
x18, y18 = ltg_ellps_lon_lat_to_fixed_grid(glm18_event_lon, glm18_event_lat, g18_nav.sat_lon, ellps_rev)

fixed_grid_locs = np.asarray([[x16, x18],
                              [y16, y18],
                             ])
goes_navs = [g16_nav, g18_nav]

final_lla, final_errs = geostationary_stereo_locate(fixed_grid_locs, goes_navs, guess_alt=12000.0)
print(final_lla, final_errs)

[ -102.15184682    33.81832317 15118.57933515] [ 1.25350989e-07 -2.13148393e-06  1.26884668e-07  2.15270135e-06]
[ -102.26746246    33.74163104 25161.47613181] [ 2.34039211e-07 -3.97933529e-06  2.36822947e-07  4.01788696e-06]
